In [2]:
#%%capture
!pip install torchvision cudatoolkit=10.0 -f
!pip install -U sentence-transformers
!pip install transformers
!pip install datasets
!pip install torch


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

-f option requires 1 argument
You should consider upgrading via the '/home/studio-lab-user/.conda/envs/default/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/studio-lab-user/.conda/envs/default/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/studio-lab-user/.conda/envs/default/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/studio-lab-user/.conda/envs/default/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
import torch
from transformers import BertTokenizerFast, BertModel

from transformers import logging
logging.set_verbosity_error()

model_weights = 'bert-base-multilingual-cased'
tokenizer = BertTokenizerFast.from_pretrained(model_weights)
sample = 'oi, eu sou o goku'

tokens_words = tokenizer.tokenize(sample)
print(tokens_words)
tokens_ids = tokenizer.encode(sample)
print(tokens_ids)
{k:v for k,v in zip(tokenizer.all_special_ids, tokenizer.all_special_tokens)}

2022-06-09 03:36:15.841363: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-09 03:36:15.841417: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

['o', '##i', ',', 'eu', 'sou', 'o', 'go', '##ku']
[101, 183, 10116, 117, 14444, 20514, 183, 11783, 10853, 102]


{100: '[UNK]', 102: '[SEP]', 0: '[PAD]', 101: '[CLS]', 103: '[MASK]'}

In [4]:
sample2 = 'banco banco banco'

model = BertModel.from_pretrained(model_weights, return_dict=True)
inputs = tokenizer(sample2, return_tensors="pt")
outs = model(**inputs)
outs.keys()

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

odict_keys(['last_hidden_state', 'pooler_output'])

In [5]:
inputs

{'input_ids': tensor([[  101, 55379, 55379, 55379,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [6]:
print(outs['last_hidden_state'].size())
banco1 = outs['last_hidden_state'][:, 1]
banco2 = outs['last_hidden_state'][:, 2]
banco3 = outs['last_hidden_state'][:, 3]

torch.Size([1, 5, 768])


In [7]:
cos_sim = torch.nn.CosineSimilarity()

# n(n-1)/2 = 3(3-1)/2
cos_sim(banco1,banco2).item(), cos_sim(banco1,banco3).item(), cos_sim(banco2,banco3).item()

(0.9906638860702515, 0.9673774838447571, 0.9787045121192932)

In [8]:
Q = 'Olá, preciso pagar a parcela do meu carro'
DOC1 = 'Emissão boleto automóvel'
DOC2 = 'Melancia, Maracuja e Manga'

Q_inputs = tokenizer(Q, return_tensors="pt")
Q_outs = model(**Q_inputs)

DOC1_inputs = tokenizer(DOC1, return_tensors="pt")
DOC1_outs = model(**DOC1_inputs)

DOC2_inputs = tokenizer(DOC2, return_tensors="pt")
DOC2_outs = model(**DOC2_inputs)

Q = Q_outs['last_hidden_state'][:, 0]
DOC1 = DOC1_outs['last_hidden_state'][:, 0]
DOC2 = DOC2_outs['last_hidden_state'][:, 0]

cos_sim(Q, DOC1).item(), cos_sim(Q, DOC2).item()

(0.8946805000305176, 0.9098262190818787)

# Sentence Transformers

# <font color='darkorange'>**Problema**: </font> Criar uma representação densa de qualidade para sentenças.


### <font color='darkorange'>**Desafio**: </font> Transformers funcionam com embeddings no nível de palavra/token, não embeddings no nível de sentença. 

---

### **<font color='lightgreen'>Em qual task devemos fazer fine tuning?</font>**
> #### Natural Language Inference (**<font color='lightgreen'>NLI</font>**) é a tarefa de determinar se uma `“hipótese”` é *verdadeira* (implicação), *falsa* (contradição) ou *indeterminada* (neutra) dada uma `“premissa”`. Exemplo:

| Premissa | Label | Hipótese |  
|----------|-------|----------|
| O cientista preparou uma nova solução | implicação | O solução criada pelo cientista é  boa |  
| Um cientista alto e um baixo estão sorrindo | neutra | Cientistas riem ao ver F$_1=0$ |  
| O Haddop está lento | contradição | A query no Hadoop está executando rápida |  


#### def $:=$ [NLI](http://nlpprogress.com/english/natural_language_inference.html)


# **<font color='lightpink'>Datasets de NLI</font>**
- ### SNLI
 - #### The Stanford Natural Language Inference (SNLI) Corpus contains around 550k hypothesis/premise pairs. Models are evaluated based on accuracy. 

- #### MNLI
 - #### The Multi-Genre Natural Language Inference (MultiNLI) corpus contains around 433k hypothesis/premise pairs. It is similar to the SNLI corpus, but covers a range of genres of spoken and written text and supports cross-genre evaluation. 

- #### STSb
 - STS Benchmark comprises a selection of the English datasets used in the STS tasks organized in the context of SemEval between 2012 and 2017. The selection of datasets include text from image captions, news headlines and user forums. 






# **<font color='lightyellow'>Sentence Transfomers (SBERT)</font>**

[ArXiv link](https://arxiv.org/pdf/1908.10084.pdf)

# **<font color='darkorange'>CrossEncoder e Bi-Encoder</font>**
<img src="https://drive.google.com/uc?id=1QtAvpTlgdYnBpb3eLdXnL-Om7fFaLXVp" alt="drawing" width="700"/>


> ## **<font color='violet'>Cross Encoder</font>**
- #### <font color='lightgreen'>**Alta Precisão:**</font> os pesos das duas sentenças são otimizados juntos e a atenção é mais "*global*" (pondera as duas sentenças ao mesmo tempo) no processo de otimização. 
- #### <font color='red'>**Slow asf:**</font> para encontrar em uma coleção de $n = 10.000$ sentenças o par com a maior similaridade requer com BERT $n · (n − 1) / 2 = 49.995.000$ cálculos de inferência.


> ## **<font color='yellow'>Bi Encoder</font>**
- #### <font color='lightgreen'>**Escalável:**</font> são capazes de indexar os candidatos codificados e comparar essas representações para cada entrada, resultando em tempos de previsão rápidos. Com a mesma complexidade de agrupar $10.000$ sentenças, o tempo é reduzido de $65$ horas para $5$ segundos. 
- #### <font color='red'>**Menor precisão:**</font> o método Bi-encoders geralmente atinge desempenho inferior em comparação com o método Cross-encoders e requer uma grande quantidade de dados de treinamento. 


<img src="https://drive.google.com/uc?id=18CjUjY78W0Kwc07XUnnoRhML4x0kC0cI" alt="drawing" width="500"/>

### **<font color='white'>Correlação de Spearman:**
> #### O coeficiente $\rho$ avalia com que intensidade a relação entre duas variáveis pode ser descrita pelo uso de uma função monótona. Obs.: O Pearson avalia relações lineares e é mais sensível a outiliers do que ao $\rho$ de Spearman. Mais sobre o assunto [aqui](https://pt.wikipedia.org/wiki/Coeficiente_de_correla%C3%A7%C3%A3o_de_postos_de_Spearman)</font>

---

# **<font color='magenta'>Principal Resultado:**
<img src="https://drive.google.com/uc?id=18fKjo9rUhmt6W06fdfqSGKKbZ4vRB8KO" alt="drawing" width="800"/>





# <font color='white'> Comparação em IR: <font color='magenta'>PESOS$_\text{ NLI}$ <font color='white'>vs <font color='lightgreen'>PESOS$_\text{ PRATELEIRA}$

In [9]:
corpus = ['Desbloquear cartão adicional',
          'Refinanciamento de parcela automóvel',
          'Seguro residencial',
          'Consórcio de veículos',
          'Redução de tarifas PJ',
          'Isenção de anuidade',
          'Contestar compras',
          'Abacaxi, bobrinha e melancia',
         ]

queries = [
           'O cartao do meu filho está bloqueado', 
           'Pagar boleto financiamento carro', 
           'Quero desconto total da tarifa do meu cartao',
          ]

In [10]:
import datasets
from sentence_transformers import util

def mean_pool(token_embeds, attention_mask):
    in_mask = attention_mask.unsqueeze(-1).float()
    pool = token_embeds * in_mask
    pool = pool.sum(1)
    return torch.nn.functional.normalize(pool, p=2, dim=-1)

tokenizer = BertTokenizerFast.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased', return_dict=True)

data_corpus = datasets.Dataset.from_dict({'corpus': corpus})
data_corpus = data_corpus.map(
    lambda x: tokenizer(
            x['corpus'], max_length=48, padding='max_length',
            truncation=True, add_special_tokens=False), batched=True)
data_corpus = data_corpus.remove_columns(['corpus', 'token_type_ids'])
data_corpus.set_format(type='torch', output_all_columns=True)
loader_corpus = torch.utils.data.DataLoader(
    data_corpus, 
    )
embds_corpus = torch.zeros(len(data_corpus)*model.config.hidden_size).view(len(data_corpus), model.config.hidden_size)
for i, b in enumerate(loader_corpus):
    o = model(b['input_ids'])
    pool = mean_pool(o['last_hidden_state'], b['attention_mask'])
    embds_corpus[i] = pool

# ------------------------------------------------------------
data_queries = datasets.Dataset.from_dict({'queries': queries})
data_queries = data_queries.map(
    lambda x: tokenizer(
            x['queries'], max_length=48, padding='max_length',
            truncation=True, add_special_tokens=False), batched=True)
data_queries = data_queries.remove_columns(['queries', 'token_type_ids'])
data_queries.set_format(type='torch', output_all_columns=True)
loader_queries = torch.utils.data.DataLoader(
    data_queries, 
    )
embds_queries = torch.zeros(len(data_queries)*model.config.hidden_size).view(len(data_queries), model.config.hidden_size)
for i, b in enumerate(loader_queries):
    o = model(b['input_ids'])
    pool = mean_pool(o['last_hidden_state'], b['attention_mask'])
    embds_queries[i] = pool

K = 5

top_k = min(K, len(corpus))
for j, query in enumerate(queries):
    cos_scores = util.pytorch_cos_sim(embds_queries[j], embds_corpus)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    print(f"\n",'- '*35)
    print(f"Query {j}: {query}")
    print(f"  >>> Top {K} most similar sentences in corpus:")
    for score, idx in zip(top_results[0], top_results[1]):
        print(f'\t{corpus[idx]:>39} -- Score: {score:.4f}')
print(f"\n",'- '*35)        

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]


 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Query 0: O cartao do meu filho está bloqueado
  >>> Top 5 most similar sentences in corpus:
	           Desbloquear cartão adicional -- Score: 0.6394
	           Abacaxi, bobrinha e melancia -- Score: 0.5127
	                     Seguro residencial -- Score: 0.5077
	                  Consórcio de veículos -- Score: 0.5020
	                    Isenção de anuidade -- Score: 0.4791

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Query 1: Pagar boleto financiamento carro
  >>> Top 5 most similar sentences in corpus:
	           Desbloquear cartão adicional -- Score: 0.6558
	                     Seguro residencial -- Score: 0.5837
	                      Contestar compras -- Score: 0.5707
	                  Consórcio de veículos -- Score: 0.5691
	   Refinanciamento de parcela automóvel -- Score: 0.5520

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Query 2: Quero descon

In [11]:
from sentence_transformers import SentenceTransformer

path_base = './bertimbau/'

embedder = SentenceTransformer(path_base)

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

top_k = min(K, len(corpus))
for j, query in enumerate(queries):
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    print(f"\n",'- '*35)
    print(f"Query {j}: {query}")
    print(f"  >>> Top {K} most similar sentences in corpus:")
    for score, idx in zip(top_results[0], top_results[1]):
        print(f'\t{corpus[idx]:>39} -- Score: {score:.4f}')
print(f"\n",'- '*35)        


 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Query 0: O cartao do meu filho está bloqueado
  >>> Top 5 most similar sentences in corpus:
	           Desbloquear cartão adicional -- Score: 0.4627
	                     Seguro residencial -- Score: 0.2947
	                      Contestar compras -- Score: 0.2739
	                    Isenção de anuidade -- Score: 0.2650
	   Refinanciamento de parcela automóvel -- Score: 0.2484

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Query 1: Pagar boleto financiamento carro
  >>> Top 5 most similar sentences in corpus:
	   Refinanciamento de parcela automóvel -- Score: 0.6649
	                     Seguro residencial -- Score: 0.5533
	           Desbloquear cartão adicional -- Score: 0.5173
	                    Isenção de anuidade -- Score: 0.4981
	                      Contestar compras -- Score: 0.4849

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Query 2: Quero descon